In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelWithLMHead
import torch
from transformers import BertModel, BertConfig

#tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = BertModel.from_pretrained('bert-base-multilingual-cased')

# taken from https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")
input_context = "The dog"
# encode input context
# input_ids = tokenizer(input_context, return_tensors="pt").to(device).input_ids
# generate 3 candidates using sampling
model = model.to(device)
#outputs = model.generate(input_ids=input_ids, max_length=20, num_return_sequences=3, do_sample=True)
#print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

#for line in tokenizer.batch_decode(outputs, skip_special_tokens=True):
    #print(line)

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
file1 = open('datasets/2021_02_27_base', 'r')
line = file1.readlines()

print(line[0])

Marie de Magdala entre et traverse le devant de la scène. Elle aperçoit les journalistes, stoppe, se couvre le visage et fait demi-tour pour sortir.



In [5]:
tokenizer.pad_token = tokenizer.eos_token
encoded_input = tokenizer(line, padding=True, truncation=True, return_tensors="pt").to(device)
# print(encoded_input)

from datasets import load_dataset

def encode(batch):
    print(batch)
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_dataset = load_dataset('text', data_files='datasets/2021_02_27_base').map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
# dataset also needs transform and not just text

tokenized_dataset["train"][1]




Using custom data configuration default-89a14e45092c0405
Reusing dataset text (/home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691)
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-29bfa8cecbc4750b.arrow
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-51de7f5d38d8d4c8.arrow
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-12100bb4b92a71b9.arrow
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f161

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [6]:
#block_size = tokenizer.model_max_length # might too big for gpu memory
block_size = 128
print(block_size)

128


In [7]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-3dea1a9667fbb194.arrow
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-0b1e6415e9349bba.arrow
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-7caa3b367b6d6da5.arrow
Loading cached processed dataset at /home/datascience/.cache/huggingface/datasets/text/default-89a14e45092c0405/0.0.0/44d63bd03e7e554f16131765a251f2d8333a5fe8a73f6ea3de012dbc49443691/cache-2fc5b9cd965c5f11.arrow


In [9]:
tokenizer.decode(lm_datasets["train"][3]["input_ids"])

'<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext

In [12]:
from transformers import Trainer, TrainingArguments

model.train()

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    remove_unused_columns = False, 
    logging_dir='./logs',            # directory for storing logs
    # evaluation_strategy = "epoch",
    learning_rate=2e-5
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,
    tokenizer=tokenizer,# training arguments, defined above
    train_dataset=lm_datasets['train']         # training dataset
)

In [13]:
trainer.train()

Step,Training Loss
500,0.502400
1000,0.485500
1500,0.459700


TrainOutput(global_step=1659, training_loss=0.47892655458272115, metrics={'train_runtime': 1248.1776, 'train_samples_per_second': 1.329, 'total_flos': 1668531650494464, 'epoch': 3.0})

In [15]:
trainer.save_model("models/first_test")

In [23]:
model2 = AutoModelForCausalLM.from_pretrained("models/first_test")
input_context = "Qui est le frère? Lui"
# encode input context
input_ids = tokenizer(input_context, return_tensors="pt").to(device).input_ids
# generate 3 candidates using sampling
model2 = model2.to(device)
outputs = model.generate(input_ids=input_ids, max_length=100, pad_token_id=50256, num_return_sequences=5, do_sample=True)
print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

for line in tokenizer.batch_decode(outputs, skip_special_tokens=True):
    print(line)

Generated: ['Qui est le frère? Lui que de se troubloir nous pas', 'Qui est le frère? Lui – doit seule dit : (Sale) Yous rien proucho que, l’esprit, et c’est qu’il ne s’ançée! J’ai vous qui puissages et jamais d’être ne sages pas, c’est nous s’ament. Il n’est pas d’à qu', 'Qui est le frère? Lui, le sagesse du Soi, tu, du soi de la formité de Suprême Absolu, avec ne le Suprême Absolu (Samuel). Tu connais le Suprême Absolu, et de la vie ou être Absolu. ', 'Qui est le frère? Lui avec le mental (Brahma, Lāyāya) est le mental. Il me s‟austre. ', 'Qui est le frère? Lui est notre, sans la châmin est quel que nous mais celle de cette voie']
Qui est le frère? Lui que de se troubloir nous pas
Qui est le frère? Lui – doit seule dit : (Sale) Yous rien proucho que, l’esprit, et c’est qu’il ne s’ançée! J’ai vous qui puissages et jamais d’être ne sages pas, c’est nous s’ament. Il n’est pas d’à qu
Qui est le frère? Lui, le sagesse du Soi, tu, du soi de la formité de Suprême Absolu, avec ne le Suprême A